In [32]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical



In [33]:
def ela(image_path, scale=50):
    original_image = cv2.imread(image_path)
    resized_image = cv2.resize(original_image, (scale, scale))
    cv2.imwrite("temp.jpg", resized_image, [cv2.IMWRITE_JPEG_QUALITY, 90])
    temp_image = cv2.imread("temp.jpg")
    ela_image = cv2.absdiff(resized_image, temp_image)
    os.remove("temp.jpg")
    return ela_image


In [34]:
def load_data(data_folder):
    data = []
    labels = []

    for image_file in os.listdir(data_folder):
        if image_file.endswith(".jpg"):
            image_path = os.path.join(data_folder, image_file)
            ela_img = ela(image_path)
            data.append(ela_img)
            labels.append("real" if "real" in image_file else "fake")

    le = LabelEncoder()
    labels = le.fit_transform(labels)
    labels = to_categorical(labels)

    return np.array(data), np.array(labels)


In [35]:
data_folder = "D:/final_dataset"
data, labels = load_data(data_folder)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.30, random_state=2)

In [37]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))  # 2 classes: real and fake
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [38]:
# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model


Epoch 1/30
88/88 [==============================] - 5s 51ms/step - loss: 2.9877 - accuracy: 0.5518 - val_loss: 0.6720 - val_accuracy: 0.5892
Epoch 2/30
88/88 [==============================] - 5s 55ms/step - loss: 0.6380 - accuracy: 0.6332 - val_loss: 0.6515 - val_accuracy: 0.6275
Epoch 3/30
88/88 [==============================] - 4s 49ms/step - loss: 0.5502 - accuracy: 0.7486 - val_loss: 0.6346 - val_accuracy: 0.6425
Epoch 4/30
88/88 [==============================] - 4s 49ms/step - loss: 0.4385 - accuracy: 0.8257 - val_loss: 0.6281 - val_accuracy: 0.6475
Epoch 5/30
88/88 [==============================] - 4s 48ms/step - loss: 0.3042 - accuracy: 0.9046 - val_loss: 0.6728 - val_accuracy: 0.6442
Epoch 6/30
88/88 [==============================] - 4s 50ms/step - loss: 0.1796 - accuracy: 0.9643 - val_loss: 0.7691 - val_accuracy: 0.6608
Epoch 7/30
88/88 [==============================] - 5s 51ms/step - loss: 0.1169 - accuracy: 0.9829 - val_loss: 0.9044 - val_accuracy: 0.6325
Epoch 8/30
88

In [39]:

accuracy = model.evaluate(X_test, y_test)[1]
print(f"Model Accuracy: {accuracy}")


38/38 [==============================] - 0s 9ms/step - loss: 1.7201 - accuracy: 0.6483
Model Accuracy: 0.6483333110809326


In [40]:
model.save("our_model.h5")

c:\Users\Rohan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
